In [29]:
import matplotlib
%matplotlib qt
from matplotlib.patches import Circle
import scipy.sparse as sp
import scipy.sparse.linalg as spl
import numpy as np
import matplotlib.pylab as plt

In [30]:
dx = dy = 0.05
dt = 0.1
D = 100
k = 1000
x = np.arange(0,1,dx)
y = np.arange(0,1,dy)
X,Y = np.meshgrid(x,y)

In [31]:
nx = x.size
ny = y.size
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs) - k*dx*dy
offsets = [-nx, -1, 0, 1, nx]
print(f"{nx=} {ny=}")

nx=20 ny=20


# Build reaction diffusion matrix and test 

In [32]:
M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')
for j in range(y.size):
    for i in range(x.size):
        cell = j*nx + i
        if i==0:
            M[cell,cell] += coeffs[1]
            if cell-1 >= 0:
                M[cell, cell-1] = 0.0
        if i==nx-1:
            M[cell, cell] += coeffs[3]
            if cell+1 < M.shape[1]:
                M[cell, cell+1] = 0.0
        if j==0:
            M[cell, cell] += coeffs[0]
        if j==ny-1:
            M[cell, cell] += coeffs[4]


In [33]:
# u = np.zeros(X.size)
deletecells = sp.eye(M.shape[0]).tolil()
# for j, yj in enumerate(y):
#     for i, xi in enumerate(x):
#         if (xi-0.5)**2 + (yj-0.5)**2 < 0.2**2:
#             cell = i + j*nx
#             deletecells[cell, cell] = 0.0
#             u[cell]= 1.0
u = np.zeros_like(X)
i0, i1 = int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)
u[i0:i1, j0:j1] = 1.0
u = u.reshape((-1,))
for i in np.nonzero(u):
    deletecells[i,i] = 0

print(f"{u.mean()=}, {u.min()=}, {u.max()=} {u.sum()=}")
fig, ax = plt.subplots()
c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
fig.colorbar(c, ax=ax)
plt.show()

u.mean()=0.075, u.min()=0.0, u.max()=1.0 u.sum()=30.0


In [34]:
M = deletecells.dot(M)
A = -dt*M + sp.eye(M.shape[0])
A = A.tocsr()
for k in range(10):
    u = spl.spsolve(A, u)
    print(f"{u.mean()=}, {u.min()=}, {u.max()=}")

u.mean()=0.25494107979240516, u.min()=0.03554825417750027, u.max()=1.0
u.mean()=0.34977574471378786, u.min()=0.09238556821299153, u.max()=1.0
u.mean()=0.40961485955101734, u.min()=0.14957976964327455, u.max()=1.0
u.mean()=0.44934650488314987, u.min()=0.19718357804709719, u.max()=1.0
u.mean()=0.47620459024986217, u.min()=0.23339298138363077, u.max()=1.0
u.mean()=0.49449507388110964, u.min()=0.2597273102081619, u.max()=1.0
u.mean()=0.5069932348109165, u.min()=0.27844767185225017, u.max()=1.0
u.mean()=0.5155478068734787, u.min()=0.2915995826793335, u.max()=1.0
u.mean()=0.5214084417882587, u.min()=0.30078224755616223, u.max()=1.0
u.mean()=0.5254256458398701, u.min()=0.30717199525334304, u.max()=1.0


In [35]:
fig, ax = plt.subplots()
c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
fig.colorbar(c, ax=ax)
# circ = Circle((0.5,0.5), 0.2, fill=False, linestyle='--', linewidth=3)
# ax.add_patch(circ)
plt.show()

# Build convection matrix

In [36]:
# Corners of the vessel
i0, i1 = int(7*ny/16), int(9*ny/16)
j0, j1 = int(4*nx/16), int(12*nx/16)

In [37]:
nPts = j1-j0
nVol = nx*ny
C = sp.diags([1/dx,-1/dx], [0,1], shape=(nPts, nPts+nVol), format='lil')
C[nPts-1,nPts] = 0
C = C.tocsr() 
# Add empty rows at the bottom of C to make it the right size
indptr = C.indptr.tolist()
for i in range(nVol):
    indptr.append(indptr[-1])
C = sp.csr_matrix((C.data, C.indices, indptr), shape=(nPts+nVol, nPts+nVol))

## Build matrix equating vascular voxel concentrations to nodal concentration

In [38]:
C4 = sp.lil_matrix((nVol, nPts))
I  = np.zeros(nVol)
for node,j in enumerate(range(j0, j1)):
    for i in range(i0, i1):
        cell = i*nx+j
        C4[cell,node] = 1
        I[cell] = 1
I = sp.diags([I], [0]).tocsr()

# Check the loss
cv = np.ones(nPts) # Vascular nodal oxygen
print(np.any(I.dot(u)-C4.dot(cv)!=0.0)) # Return False if each voxel's oxygen is equal to its corresponding node's oxygen 

False


## Build mass exchange between vessels and tissue

In [39]:
NodesToEndothelialCells = sp.lil_matrix((nPts, nPts+nVol))
for node, j in enumerate(range(j0, j1)):
    for i in [i0-1,i1]:
        cell = i*nx + j
        NodesToEndothelialCells[node, node] = -1
        NodesToEndothelialCells[node, nPts+cell] = 1
for i in range(i0, i1):
    for node, j in zip([0, nPts-1],[j0-1,j1]):
        cell = i*nx + j
        NodesToEndothelialCells[node, nPts+cell] = 1
        NodesToEndothelialCells[node, node] = -1        

In [75]:
Arr = NodesToEndothelialCells.toarray()
u = np.zeros(nVol)
for j in range(j0, j1):
    for i in range(i0, i1):
        u[i*nx+j] = 1

for node in range(nPts):
    u[np.nonzero(Arr[node, nPts:])] = 2

plt.pcolormesh(u.reshape(X.shape), cmap='RdBu', edgecolor='k')

In [80]:
Gamma = sp.diags([np.arange(nPts)+1], [0], shape=(nPts, nPts))
MassExchange = NodesToEndothelialCells.T.dot(Gamma).dot(NodesToEndothelialCells)
fig, ax = plt.subplots()
c = ax.pcolormesh(abs(MassExchange.toarray()), cmap='RdBu')
fig.colorbar(c, ax=ax)

In [81]:
print((i0-1)*nx+j0, (i0-1)*nx+j1, i1*nx+j0, i1*nx+j1)
Arr = MassExchange.toarray()[nPts:, :]
for i in np.nonzero(u)[0]:
    if u[i]==2:
        nnz = np.nonzero(Arr[i,:])[0]
        if nnz.size > 0:
            print(i, nnz[0], nnz[1:]-nPts, Arr[i,:][nnz], u[i])


145 155 225 235
145 0 [145 164 184 204 225] [-1.  1.  1.  1.  1.  1.] 2.0
146 1 [146 226] [-2.  2.  2.] 2.0
147 2 [147 227] [-3.  3.  3.] 2.0
148 3 [148 228] [-4.  4.  4.] 2.0
149 4 [149 229] [-5.  5.  5.] 2.0
150 5 [150 230] [-6.  6.  6.] 2.0
151 6 [151 231] [-7.  7.  7.] 2.0
152 7 [152 232] [-8.  8.  8.] 2.0
153 8 [153 233] [-9.  9.  9.] 2.0
154 9 [154 175 195 215 234] [-10.  10.  10.  10.  10.  10.] 2.0
164 0 [145 164 184 204 225] [-1.  1.  1.  1.  1.  1.] 2.0
175 9 [154 175 195 215 234] [-10.  10.  10.  10.  10.  10.] 2.0
184 0 [145 164 184 204 225] [-1.  1.  1.  1.  1.  1.] 2.0
195 9 [154 175 195 215 234] [-10.  10.  10.  10.  10.  10.] 2.0
204 0 [145 164 184 204 225] [-1.  1.  1.  1.  1.  1.] 2.0
215 9 [154 175 195 215 234] [-10.  10.  10.  10.  10.  10.] 2.0
225 0 [145 164 184 204 225] [-1.  1.  1.  1.  1.  1.] 2.0
226 1 [146 226] [-2.  2.  2.] 2.0
227 2 [147 227] [-3.  3.  3.] 2.0
228 3 [148 228] [-4.  4.  4.] 2.0
229 4 [149 229] [-5.  5.  5.] 2.0
230 5 [150 230] [-6.  6.  6.] 

In [82]:
Arr = MassExchange.toarray()[:nPts, :]
for node in range(nPts):
    nnz = np.nonzero(Arr[node, :])[0]
    print(node, nnz, Arr[node, nnz])

0 [  0 155 174 194 214 235] [ 1. -1. -1. -1. -1. -1.]
1 [  1 156 236] [ 2. -2. -2.]
2 [  2 157 237] [ 3. -3. -3.]
3 [  3 158 238] [ 4. -4. -4.]
4 [  4 159 239] [ 5. -5. -5.]
5 [  5 160 240] [ 6. -6. -6.]
6 [  6 161 241] [ 7. -7. -7.]
7 [  7 162 242] [ 8. -8. -8.]
8 [  8 163 243] [ 9. -9. -9.]
9 [  9 164 185 205 225 244] [ 10. -10. -10. -10. -10. -10.]
